**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Project Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a project account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the project account;

6 - Queries the blockchain again to confirm the succesfull details setting.

Every account succesfully created is dumped to `datasets/projects.json` for later use and evidence collection.  

```mermaid

%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant Blockchain as "Iroha 1 Blockchain"
    participant IPFS as "Interplanetary File System"

    Note over Platform, Blockchain: Deploy smart contract for account creation
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully
    
    Note over Platform, Blockchain: Project Account creation
    Platform->>Blockchain: Create Project Account
    Blockchain-->>Platform: Project Account Created Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm account creation
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
        
    Note over Platform, Blockchain: Deploy smart contract for details setting
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully

    Note over Platform, IPFS: Set project details
    Platform->>IPFS: Send project JSON metadata to IPFS
    IPFS-->>Platform: Send metadata CID to the Platform
    Platform->>Blockchain: Set Project Metadata as Details in Blockchain
    Blockchain-->>Platform: Project Details Set Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm details setting
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
```

In [19]:
import binascii
from iroha import IrohaCrypto
from iroha import Iroha, IrohaGrpc
import sys
from Crypto.Hash import keccak
import integration_helpers
from name_generator import left, right
import json
from utilities import *
from project_id_generator import * # - Random generated synthetic project ID
import icecream as ic
from iroha_helper import create_account_contract, create_project_account, set_account_detail, create_detail_contract
from loguru import logger
from dump_to_json import ProjectAccount, update_or_append_project_metadata
from ipfs_functions import *
from synthetic_project_json_ld import *

if sys.version_info[0] < 3:
    raise Exception("Python 3 or a more recent version is required.")

# Load configuration from config.json file
config_path = "config.json"  # Update this path as needed
with open(config_path, "r") as f:
    config = json.load(f)

IROHA_HOST_ADDR = config["IROHA_HOST_ADDR"]
IROHA_PORT = config["IROHA_PORT"]
ADMIN_ACCOUNT_ID = config["ADMIN_ACCOUNT_ID"]
ADMIN_PRIVATE_KEY = config["ADMIN_PRIVATE_KEY"]
iroha = Iroha(ADMIN_ACCOUNT_ID)
net = IrohaGrpc("{}:{}".format(IROHA_HOST_ADDR, IROHA_PORT))




In [20]:
DOMAIN = "test"

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;  

In [21]:
# - Deploy contract
hash = create_account_contract()


2025-01-11 22:13:52.873 | INFO     | integration_helpers:tracer:34 - 	Entering "create_account_contract"
2025-01-11 22:13:52.895 | INFO     | iroha_helper:create_account_contract:44 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 22:13:52.898 | INFO     | iroha_helper:create_account_contract:44 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 22:13:54.512 | INFO     | iroha_helper:create_account_contract:44 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 22:13:54.542 | INFO     | iroha_helper:create_account_contract:44 - ('COMMITTED', 5, 0)
2025-01-11 22:13:54.543 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_account_contract"


2 - Creates a project account into the blockchain;

In [22]:
project_private_key = IrohaCrypto.private_key()
project_public_key = IrohaCrypto.derive_public_key(project_private_key).decode("utf-8")



# @integration_helpers.trace
# def create_project_account(address, project_account:ProjectAccount):
#     params = integration_helpers.get_first_four_bytes_of_keccak(
#         b"createAccount(string,string,string)"
#     )
#     no_of_param = 3
#     for x in range(no_of_param):
#         params = params + integration_helpers.left_padded_address_of_param(
#             x, no_of_param
#         )
#     params = params + integration_helpers.argument_encoding(project_id)  # source account id
#     params = params + integration_helpers.argument_encoding(DOMAIN)  # domain id
#     params = params + integration_helpers.argument_encoding(project_public_key)  #  key
#     tx = iroha.transaction(
#         [
#             iroha.command(
#                 "CallEngine", caller=ADMIN_ACCOUNT_ID, callee=address, input=params
#             )
#         ]
#     )
#     IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
#     response = net.send_tx(tx)
#     for status in net.tx_status_stream(tx):
#         print(status)
#     hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
#     line_number = dump_project_to_json_ld(f"{project_id}@{DOMAIN}", project_public_key) #dumps this data to dataset/projects.json for later use.
#     print(line_number)
#     return hex_hash


project_id = generate_project_id()

print(f"Creating project with name: {project_id}")

project_account = ProjectAccount(
    project_id=f"{project_id}@{DOMAIN}",
    public_key=project_public_key
)

logger.info(f"Creating project with name: {project_id}")

# - Create account
address = integration_helpers.get_engine_receipts_address(hash)
create_project_account(address, project_id, DOMAIN, project_public_key, project_account)



@integration_helpers.trace
def get_account(address):
    params = integration_helpers.get_first_four_bytes_of_keccak(b"getAccount(string)")
    no_of_param = 1
    for x in range(no_of_param):
        params = params + integration_helpers.left_padded_address_of_param(
            x, no_of_param
        )
    params = params + integration_helpers.argument_encoding(f"{project_id}@{DOMAIN}")  # project id
    tx = iroha.transaction(
        [
            iroha.command(
                "CallEngine", caller=ADMIN_ACCOUNT_ID, callee=address, input=params
            )
        ]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    response = net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    return hex_hash





# - Get account info
hash = get_account(address)
integration_helpers.get_engine_receipts_result(hash)

print("done")

2025-01-11 22:13:54.572 | INFO     | __main__:<module>:45 - Creating project with name: 77323
2025-01-11 22:13:54.573 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-11 22:13:54.595 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-11 22:13:54.597 | INFO     | integration_helpers:tracer:34 - 	Entering "create_project_account"
2025-01-11 22:13:54.613 | INFO     | iroha_helper:create_project_account:113 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 22:13:54.615 | INFO     | iroha_helper:create_project_account:113 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


Creating project with name: 77323


2025-01-11 22:13:57.567 | INFO     | iroha_helper:create_project_account:113 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 22:13:57.591 | INFO     | iroha_helper:create_project_account:113 - ('COMMITTED', 5, 0)
2025-01-11 22:13:57.593 | INFO     | dump_to_json:dump_project_to_json_ld:132 - Error appending entry to JSON-LD: 'str' object has no attribute 'project_id'
2025-01-11 22:13:57.595 | INFO     | iroha_helper:create_project_account:116 - None
2025-01-11 22:13:57.596 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_project_account"
2025-01-11 22:13:57.597 | INFO     | integration_helpers:tracer:34 - 	Entering "get_account"


('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


2025-01-11 22:14:00.659 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_account"
2025-01-11 22:14:00.660 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-11 22:14:00.681 | INFO     | integration_helpers:get_engine_receipts_result:92 -                                                                                                                               J{"account_id":"77323@test","domain_id":"test","quorum":1,"json_data":"{}"}                      
2025-01-11 22:14:00.682 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_result"


('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
done


('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
done


3 - Queries the blockchain to confirm the succesful creation of the account;

In [23]:
#Query - GetAccountDetail

query = iroha.query('GetAccountDetail',account_id=f"{project_id}@{DOMAIN}")
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response.account_detail_response
print(f'Account id = {project_id}@{DOMAIN}, details = {data.detail}')


Account id = 77323@test, details = {}


4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [24]:
# - Deploys the contract
hash = create_detail_contract()
address = integration_helpers.get_engine_receipts_address(hash)
integration_helpers.get_engine_receipts_result(hash)

2025-01-11 22:14:00.747 | INFO     | integration_helpers:tracer:34 - 	Entering "create_detail_contract"
2025-01-11 22:14:00.765 | INFO     | iroha_helper:create_detail_contract:58 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 22:14:00.768 | INFO     | iroha_helper:create_detail_contract:58 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 22:14:03.695 | INFO     | iroha_helper:create_detail_contract:58 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 22:14:03.729 | INFO     | iroha_helper:create_detail_contract:58 - ('COMMITTED', 5, 0)
2025-01-11 22:14:03.730 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_detail_contract"
2025-01-11 22:14:03.732 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-11 22:14:03.752 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-11 22:14:03.753 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-11 22:14:03.7

5 - Sets details for the project account:

In [25]:

# Example project ID base
project_id_base = project_id  # Replace with your actual base project ID

# Generate synthetic project metadata JSON (assumed to be done already)
project_metadata_json = generate_synthetic_entry_ld()
project_metadata_cid = upload_json_to_ipfs(project_metadata_json)  # Assuming this is done before calling the function

# Call the function to update or append a new entry to the JSON-LD file
updated_entry_count = update_or_append_project_metadata(
    project_id_base, 
    project_metadata_json, 
    project_metadata_cid
)

# Setting account detail using the generated project metadata CID
hash = set_account_detail(
    address, 
    f"{project_id}@{DOMAIN}",  # Project ID as account
    "project_metadata_cid",    # The key we're setting
    project_metadata_cid       # The value (CID from IPFS)
)

# Log the account ID for reference
logger.info(f"Project Account: {project_id}@{DOMAIN}")

2025-01-11 22:14:03.817 | INFO     | dump_to_json:update_or_append_project_metadata:228 - Appended new project entry for project ID: 77323@test
2025-01-11 22:14:03.820 | INFO     | dump_to_json:update_or_append_project_metadata:235 - Current total entries in 'datasets/projects.json': 2
2025-01-11 22:14:03.821 | INFO     | iroha_helper:tracer:247 - 	Entering "set_account_detail"
2025-01-11 22:14:03.837 | INFO     | iroha_helper:set_account_detail:299 - None
2025-01-11 22:14:03.839 | INFO     | iroha_helper:set_account_detail:301 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 22:14:06.757 | INFO     | iroha_helper:set_account_detail:301 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 22:14:06.787 | INFO     | iroha_helper:set_account_detail:301 - ('COMMITTED', 5, 0)
2025-01-11 22:14:06.789 | INFO     | iroha_helper:tracer:249 - 	Leaving "set_account_detail"
2025-01-11 22:14:06.790 | INFO     | __main__:<module>:24 - Project Account: 77323@test


2025-01-11 21:56:46.347 | INFO     | dump_to_json:update_or_append_project_metadata:235 - Current total entries in 'datasets/projects.json': 8
2025-01-11 21:56:46.348 | INFO     | iroha_helper:tracer:247 - 	Entering "set_account_detail"
2025-01-11 21:56:46.356 | INFO     | iroha_helper:set_account_detail:299 - None
2025-01-11 21:56:46.357 | INFO     | iroha_helper:set_account_detail:301 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 21:56:49.206 | INFO     | iroha_helper:set_account_detail:301 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 21:56:49.233 | INFO     | iroha_helper:set_account_detail:301 - ('COMMITTED', 5, 0)
2025-01-11 21:56:49.236 | INFO     | iroha_helper:tracer:249 - 	Leaving "set_account_detail"
2025-01-11 21:56:49.237 | INFO     | __main__:<module>:96 - Project Account: 87550@test


6 - Queries the blockchain to confirm the succesful details setting

In [26]:
#Query - GetAccountDetail

query = iroha.query('GetAccountDetail',account_id=f"{project_id}@{DOMAIN}")
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
data = response.account_detail_response
details = data.detail
logger.info(f'Project Account id = {project_id}@{DOMAIN}, {details}')


2025-01-11 22:14:06.820 | INFO     | __main__:<module>:8 - Project Account id = 77323@test, { "admin@test" : { "project_metadata_cid" : "QmaWfoLPTka6WB8qTFiJ7YhZs2k3XPkdDDocEmTS5L7ZF7" } }


7 - Retrieves User JSON-LD from IPFS

In [27]:
# Convert the JSON string to a Python dictionary
details_dict = json.loads(details)

# Now you can access the specific key like this
project_metadata_cid = details_dict["admin@test"]["project_metadata_cid"]

logger.info(f"Project Metadata CID: {project_metadata_cid}")

project_metadata = download_json_from_ipfs(project_metadata_cid)

logger.info(f"Projecte Metadata: {project_metadata}")

2025-01-11 22:14:06.830 | INFO     | __main__:<module>:7 - Project Metadata CID: QmaWfoLPTka6WB8qTFiJ7YhZs2k3XPkdDDocEmTS5L7ZF7
2025-01-11 22:14:06.838 | INFO     | __main__:<module>:11 - Projecte Metadata: {'@context': {'dc': 'http://purl.org/dc/terms/', 'schema': 'http://schema.org/'}, '@type': 'schema:ResearchProject', 'dc:abstract': 'This paper analyzes how gene therapy influences energy storage, providing insights into how to maximize its personalized medicine.', 'schema:endDate': '2028-06-19', 'schema:funding': {'@type': 'schema:Organization', 'schema:name': 'National Geographic Society'}, 'schema:keywords': ['gene therapy', 'energy storage', 'personalized medicine'], 'schema:location': {'@type': 'schema:Place', 'schema:name': 'Accra, Ghana'}, 'schema:name': 'Analyzing the Influence of gene therapy on energy storage', 'schema:startDate': '2018-10-29'}
